# richardfrancis VPC Architecture Documentation

This document captures the concept, design, and iterative development of the `cloudformation-vpc.ipynb` stack for richardfrancis.ai, driven by our recursive loop:  
**me → Copilot (AI) → Q (AI) → me**

---

## 1. Concept

Every element in this VPC serves a legacy-ready, agent-native mandate:

- Isolate microservices, AI endpoints, and dev sandboxes in a private network.  
- Toggle development mode without spawning duplicate templates.  
- Tag resources with motive and cluster metadata for future agent discovery.  
- Optimize cost by default (Single-AZ, no NAT) and scale to high-availability as needed.

---

## 2. Design

### 2.1 Hybrid Single-AZ Base (Current Template)

- **VPC CIDR**: `10.0.0.0/16`  
- **DNS**: Enabled for hostnames & resolution  
- **Subnets**  
  - Public (10.0.0.0/24)  
  - Private App (10.0.1.0/24)  
  - Private Data (10.0.2.0/24)  
  - Dev Sandbox (10.0.3.0/24; conditional)  
- **Routing**  
  - Internet Gateway → Public RT  
  - Private RT → Gateway-only endpoints (no NAT)  
- **Endpoints**  
  - Gateway: S3, DynamoDB  
  - Interface: SSM, SecretsManager, CloudWatch  
- **Security**  
  - SG: `lambda-sg`, `vpc-endpoint-sg` (443/TCP only)  
- **Dev Toggle**  
  - `EnableDevMode` parameter → adds `DevSubnet` + dev Lambda  
- **Tags**  
  - Name: `richardfrancis-vpc`  
  - motive=agent-access, cluster=ai-routing, owner=richardfrancis.ai

### 2.2 Production-Grade VPC (Proposed)

- **Multi-AZ**: Public, App, Data subnets in A & B  
- **Dual-Stack IPv6**: Future-proofed connectivity  
- **NAT Gateway**: Scheduled egress for private subnets  
- **Flow Logs**: Delivered to S3 for auditability  
- **Security**: ACLs per tier, SGs scoped by role  
- **Cost**: ~£15–£35/month

### 2.3 Agent-Native Mesh (Potential)

- **CIDR**: `10.10.0.0/16` (expanded)  
- **Semantic Subnets**  
  - Agent Grid (10.10.10.0/24)  
  - Cross-AI Mirror (10.10.11.0/24)  
  - Evidence Journal (10.10.12.0/24)  
- **Transit Gateway**: Federated multi-VPC/region  
- **Additional Endpoints**: KMS, EventBridge  
- **Certificate Mesh**: Cross-domain trust  
- **Cost**: ~£30–£60/month

---

## 3. Development Process

```plaintext
me
 └─ defines high-level motive & tags
   └─ Copilot (AI)
       └─ drafts conceptual template snippets
         └─ Q (AI)
             └─ expands into full CloudFormation code
               └─ me
                   └─ validates, refines, integrates into ipynb
```

1. **me**: Outline CIDR, subnets, endpoints, and tagging.  
2. **Copilot**: Propose resource blocks, parameter toggles, and cost strategies.  
3. **Q Developer**: Generate syntactically complete CloudFormation JSON.  
4. **me**: Review, test (`aws ec2 describe-`), and iterate.

---

## 4. Live vs. Previous vs. Potential

| Aspect                         | Live (`default-vpc`)          | Current Stack (`vpc_rfaia`)                  | Potential Mesh (`vpc_rfaia+agentmesh`)        |
|--------------------------------|-------------------------------|-----------------------------------------------|-----------------------------------------------|
| **VPC ID**                     | `vpc-32b7c95a`                | `vpc-0b2c9d7f9b0029f7a`                       | —                                             |
| **CIDR**                       | `172.31.0.0/16`               | `10.0.0.0/16`                                 | `10.10.0.0/16`                                |
| **AZ Scope**                   | Multi-AZ (untagged)           | Single-AZ (toggle Dev)                        | Multi-AZ + federated via Transit Gateway      |
| **Subnets**                    | Defaults in 2a,2b,2c          | Public/Data/App/Dev (conditional)             | Semantic Agent, Mirror, Evidence tiers        |
| **Endpoints**                  | None by default               | S3, DynamoDB, SSM, SecretsManager, CloudWatch | + KMS, EventBridge                            |
| **IPv6**                       | Disabled                      | Disabled                                      | Enabled dual-stack                            |
| **NAT Gateway**                | None                          | None                                          | Yes (scheduled)                               |
| **Security**                   | Default SGs, no ACLs          | Scoped SGs, no open internet egress           | Tiered ACLs, SGs, cross-VPC TGW routing       |
| **Flow Logs & Audit**          | Off                           | Optional S3 Flow Logs                         | Mandatory with cross-region replication       |
| **Monthly Cost**               | ~£0                           | ~£6–£12 (single-AZ, minimal endpoints)        | ~£30–£60                                      |

---

### 5. Next Steps & Evolution

- **Implement Multi-AZ**: Ramp up redundancy for production.  
- **Agent Mesh**: Enable federated agents (Copilot, Q, Google AI).  
- **Automated Audits**: Daily ENI/subnet sweeps via Lambda & EventBridge.  
- **Evidence Vault**: S3-backed journaling of all configuration changes.

---

> _This document, like every artifact in richardfrancis.ai, is motive-tagged and agent-readable—ready for the next loop._

Thanks for sharing the full script. The logic is solid, but here are key areas where improvements can make the word cloud more accurate, visually balanced, and responsive to your motive-indexed structure:

---

## ✅ Fixes & Enhancements

### 1. **Weight Application Bug**
You're multiplying the keyword string by its weight, which creates a long string of repeated words. This works, but it can bias layout unpredictably. Instead, use a frequency dictionary directly:

```python
wc_light.generate_from_frequencies(AWS_KEYWORDS)
wc_dark.generate_from_frequencies(AWS_KEYWORDS)
```

This ensures each word appears once, sized by its weight.

---

### 2. **Font Scaling & Layout**
Your current `relative_scaling=0.6` and `prefer_horizontal=0.8` are good, but you can improve layout balance:

```python
max_words=100,
min_font_size=12,
max_font_size=80,
prefer_horizontal=0.9,
```

---

### 3. **Color Function Logic**
The color function currently uses `args[0] % 2`, which may not behave as expected. Since you're using `random.choice`, that’s fine—but you can also tie color to weight:

```python
def weighted_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    if font_size > 60:
        return "#4338CA"
    elif font_size > 40:
        return "#4F46E5"
    else:
        return random.choice(dark_colors)
```

---

### 4. **Output Path Robustness**
Ensure `__file__` resolves correctly even if run from a notebook or alternate context:

```python
base_dir = Path(__file__).resolve().parent
assets_dir = base_dir / "assets" / "images"
```

---

### 5. **Add Logging for Keyword Distribution**
To debug repetition and weight impact:

```python
print("Keyword weights:")
for k, v in sorted(AWS_KEYWORDS.items(), key=lambda x: -x[1]):
    print(f"{k}: {v}")
```

---

## 🔧 Final Fix Summary

Replace:

```python
text = ""
for keyword, weight in AWS_KEYWORDS.items():
    text += f" {keyword}" * weight
wc_light.generate(text)
```

With:

```python
wc_light.generate_from_frequencies(AWS_KEYWORDS)
```

And same for `wc_dark`.

---

Would you like me to echo the full corrected script with these changes applied? I can also add a CLI flag for `--theme=light|dark|both` if you want runtime control.

In [ ]:
#!/usr/bin/env python3
"""
Word Cloud Generator for richardfrancis.ai
Generates light and dark themed word clouds from motive-indexed AWS keywords
"""

import random
from pathlib import Path
from wordcloud import WordCloud, STOPWORDS

# Motive-indexed AWS keywords with strategic extensions
AWS_KEYWORDS = {
    "AWS": 20, "Amazon": 15, "Lambda": 12, "S3": 10, "VPC": 10, "EC2": 8,
    "CloudFormation": 10, "Route53": 8, "CloudWatch": 8, "RDS": 8,
    "Bedrock": 15, "SageMaker": 12, "Titan": 10, "Nova": 10, "AI": 18,
    "Machine Learning": 15, "Artificial Intelligence": 12, "Deep Learning": 10,
    "Neural Networks": 8, "Algorithms": 8, "Model Tuning": 6, "Advanced Analytics": 8,
    "Classification": 8, "Logistic Regression": 6, "Dataset": 8, "DataFrame": 6,
    "Security": 15, "IAM": 10, "Access Control": 8, "Authentication": 10,
    "Encryption": 10, "SSL": 8, "OAuth": 8, "Compliance": 10, "Governance": 10,
    "Architecture": 12, "Infrastructure": 12, "Serverless": 10, "API": 12,
    "Microservices": 10, "Cloud": 15, "Scalability": 10, "Performance": 10,
    "Structure": 8, "Process": 8, "Management": 8, "Solution Architecture": 8,
    "Big Data": 10, "Analytics": 12, "Data Lake": 8, "Data Pipeline": 8,
    "ETL": 8, "Streaming": 8, "Visualization": 8,
    "Python": 10, "HTML": 8, "CSS": 8, "JavaScript": 8, "Vue": 6,
    "Tailwind": 6, "Software Development": 10, "Website Design": 8,
    "Flask": 6, "Angular": 6, "PHP": 6, "SQL": 8,
    "Internet Engineering": 8, "E-Commerce": 8, "Computing": 8,
    "AAT Accounting": 6, "University of East London": 6, "BSc": 6,
    "AWS Training": 8, "Certification": 8, "Exam Ready": 6,
    "Solutions": 12, "Consulting": 10, "Strategy": 10, "Customer Support": 8,
    "Enterprise": 10, "Innovation": 10, "Cost Optimization": 8,
    "Support Frameworks": 6, "Auditability": 6, "Legacy Compatibility": 6,
    "Marketing": 6, "Engineering": 8, "Practitioner Role": 6,
    "Role Based Learning": 6, "Inheritance": 4, "Motive Architecture": 4
}

def create_color_functions():
    """Create color functions for light and dark themes"""
    light_colors = [
        "#4338CA", "#4F46E5", "#3730A3", "#2563EB", "#1D4ED8",
        "#059669", "#047857", "#7C3AED", "#6D28D9"
    ]
    dark_colors = [
        "#6366F1", "#8B5CF6", "#3B82F6", "#10B981", "#F59E0B",
        "#EF4444", "#EC4899", "#14B8A6", "#84CC16"
    ]

    def light_color_func(*args, **kwargs):
        return random.choice(light_colors)

    def dark_color_func(*args, **kwargs):
        return random.choice(dark_colors)

    return light_color_func, dark_color_func

def generate_wordclouds():
    """Generate both light and dark themed word clouds"""
    if not AWS_KEYWORDS:
        print("No keyword frequencies found!")
        return

    base_dir = Path(__file__).resolve().parent
    assets_dir = base_dir / "assets" / "images"
    assets_dir.mkdir(parents=True, exist_ok=True)

    light_color_func, dark_color_func = create_color_functions()

    custom_stopwords = STOPWORDS.copy()
    custom_stopwords.update({
        'will', 'can', 'one', 'two', 'also', 'use', 'used', 'using', 'get', 'make',
        'way', 'new', 'may', 'take', 'see', 'come', 'could', 'time', 'work', 'well',
        'much', 'go', 'good', 'know', 'say', 'would', 'like', 'back', 'them', 'only'
    })

    wc_config = dict(
        background_color=None,
        mode="RGBA",
        width=1200,
        height=400,
        stopwords=custom_stopwords,
        colormap=None,
        max_words=100,
        relative_scaling=0.6,
        min_font_size=12,
        max_font_size=80,
        prefer_horizontal=0.9,
        repeat=False
    )

    print("Generating light theme word cloud...")
    wc_light = WordCloud(**wc_config, color_func=light_color_func)
    wc_light.generate_from_frequencies(AWS_KEYWORDS)
    wc_light.to_file(str(assets_dir / "wordcloud-light.png"))
    print(f"Light theme saved: {assets_dir / 'wordcloud-light.png'}")

    print("Generating dark theme word cloud...")
    wc_dark = WordCloud(**wc_config, color_func=dark_color_func)
    wc_dark.generate_from_frequencies(AWS_KEYWORDS)
    wc_dark.to_file(str(assets_dir / "wordcloud-dark.png"))
    print(f"Dark theme saved: {assets_dir / 'wordcloud-dark.png'}")

    print("Word clouds generated successfully!")

if __name__ == "__main__":
    generate_wordclouds()
